This code produces maps for a comparison of impact of potato and wheat.

In [17]:
import os
import mypackages.myrasters as mr
import numpy as np
import fiona
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline

In [18]:
data_dir = os.path.join('..', 'data')
cf_dir = os.path.join('..', 'output/characterization_factors')
out_dir = os.path.join('..', 'output/impact')

In [19]:
shapefile_name = 'shapefiles/countries.shp'
shapefile = os.path.join(data_dir, shapefile_name)

Read in the ton-kilometers per crop:

In [20]:
tkm = pd.read_csv(os.path.join(data_dir, 'inventory/tkm_inventory.csv'), index_col=0)

Read in the characterization factor rasters:

In [21]:
ts_raster = mr.MyRaster(os.path.join(cf_dir, 'ts_100yrs.tif'))
ms_raster = mr.MyRaster(os.path.join(cf_dir, 'ms_100yrs.tif'))
bs_raster = mr.MyRaster(os.path.join(cf_dir, 'bs_100yrs.tif'))

Define a function to calculate the total impact:

In [22]:
def total_impact_global_raster(crop):
        
    ts_array = ts_raster.get_array()
    ts = tkm[crop]['top soil'] * ts_array
    
    ts_array = None
    
    ms_array = ms_raster.get_array()
    ms = tkm[crop]['mid soil'] * ms_array
    
    ms_array = None
    
    impact = ts + ms
    
    ts = None
    ms = None
    
    bs_array = bs_raster.get_array()
    bs = tkm[crop]['bottom soil'] * bs_array
    
    impact = impact + bs
    
    impact[bs_array == bs_raster.NoDataValue] = bs_raster.NoDataValue
    
    bs_array = None
    bs = None
    
    impact = mr.mask_array(impact, bs_raster.NoDataValue)
    
    return impact


Read in crops:

In [23]:
[col for col in tkm.columns if 'Potato' in col]

['Potatoes, OeLN intensive, wholesale',
 'Potatoes, OeLN, wholesale',
 'Potatoes, organic, wholesale',
 'Potatoes, OeLN intensive, retail',
 'Potatoes, OeLN, retail',
 'Potatoes, organic, retail']

In [24]:
[col for col in tkm.columns if 'wheat' in col]

['Winter wheat TOP, OeLN intensive, wholesale',
 'Winter wheat TOP, OeLN extensive, wholesale',
 'Winter wheat TOP, organic, wholesale',
 'Winter wheat TOP, OeLN extensive, retail',
 'Winter wheat TOP, organic, retail',
 'Summer wheat TOP, OeLN intensive, wholesale',
 'Summer wheat TOP, OeLN extensive, wholesale',
 'Summer wheat TOP, organic, wholesale']

Run calculation:

In [25]:
%%time

potato = total_impact_global_raster('Potatoes, OeLN intensive, wholesale')
wheat = total_impact_global_raster('Winter wheat TOP, OeLN intensive, wholesale')

CPU times: user 1min 24s, sys: 50.1 s, total: 2min 14s
Wall time: 2min 13s


In [26]:
np.percentile(potato, 98)

0.41940131783485413

In [ ]:
vmin = 0
vmax = 0.5

### Plot results:

In [ ]:
gdf = gpd.read_file(shapefile)

In [ ]:
def createColourbar(lwr, upr):
    """Create a colourbar with limits of lwr and upr"""
    divider = make_axes_locatable(plt.gca())
    cax = divider.append_axes("right", size="2%", pad=0.1)
    norm = matplotlib.colors.Normalize(vmin = lwr, vmax = upr, clip = False)

    c = matplotlib.colorbar.ColorbarBase(cax, cmap='YlOrRd', norm=norm)
    return c

In [ ]:
%%time

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(111)

ax.set_xlim([-180, 180])
ax.set_ylim([-56, 84])
ax.set_xticks([])
ax.set_yticks([])
ax.set_frame_on(False)

gdf.plot(ax=ax, facecolor='none', edgecolor='#000000', linewidths=0.3)

ax.imshow(potato, extent=[-180, 180, -56, 84], cmap='YlOrRd', vmin=vmin, vmax=vmax)

norm = matplotlib.colors.Normalize(vmin, vmax, clip = False)

cax = fig.add_axes([0.2, 0.25, 0.01, 0.3]) # xmin, ymin, dx, dy
matplotlib.colorbar.ColorbarBase(cax, cmap='YlOrRd', norm=norm)
cax.yaxis.set_ticks_position('left')

plt.savefig(os.path.join(out_dir, 'impact_potato.png'),
            dpi=300, bbox_inches='tight', pad_inches=0.1)

In [ ]:
%%time

fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(111)

ax.set_xlim([-180, 180])
ax.set_ylim([-56, 84])
ax.set_xticks([])
ax.set_yticks([])
ax.set_frame_on(False)

gdf.plot(ax=ax, facecolor='none', edgecolor='#000000', linewidths=0.3)

ax.set_xlim(-180, 180)
ax.set_ylim(-56, 84)

ax.imshow(wheat, extent=[-180, 180, -56, 84], cmap='YlOrRd', vmin=vmin, vmax=vmax)

plt.savefig(os.path.join(out_dir, 'impact_wheat.png'),
            dpi=300, bbox_inches='tight', pad_inches=0.1)

In [ ]:
out_filename = 'impact_potato'
mr.array2geotiff_rastercopy(potato.data, os.path.join(out_dir, out_filename), bs_raster.raster)